In [1]:
import os
import pickle as pkl

In [2]:
pkls = [f for f in os.listdir('./') if '.pkl' in f]

In [3]:
sparse_weights = {}
cnt = 0
for p in pkls:
    with open(p, 'rb') as f:
        weight = pkl.load(f).detach().numpy()
        
    _, topbot, m, sparsity = p.split('_')
    sparsity = sparsity.split('.pkl')[0]
    
    if sparse_weights.get(sparsity) == None:
        sparse_weights[sparsity] = {
            "botl.0": None,
            "botl.2": None,
            "botl.4": None,
            "topl.0": None,
            "topl.2": None,
            "topl.4": None,
            "topl.6": None,
            "topl.8": None,
        }
    
    sparse_weights[sparsity][topbot+m] = weight

In [4]:
# handpicked
sparsity_to_test = [
    '0.5045721601363784',
    '0.5541957834992964',
    '0.599990952078147',
    '0.6509844477216149',
    '0.7006824838997727',
    '0.7513115258686005',
    '0.8001901754789479',
    '0.8511912815239745',
    '0.9004462232113866',
    '0.9499996617599308'
]

# Make dest file
def create_dir(dest_dir):
    try:
        os.stat(dest_dir)
    except:
        os.makedirs(dest_dir, exist_ok=True)


In [10]:
dest_dir = 'sparse_weights/ts_95/'

def dump_to_txt(numpy_weights, filepath):
    # Should be 2-dim array
    assert len(numpy_weights.shape) == 2
    
    with open(filepath, "w") as f:
        f.write("{} {}\n".format(*numpy_weights.shape))
        f.writelines(
            [" ".join([str(i) for i in l]) + "\n" for l in numpy_weights]            
        )
    print("dumped: {}".format(filepath))

for sp in sparsity_to_test:
    dest_path = os.path.join(dest_dir, "sparsity_{}".format(round(float(sp), 2)))
    create_dir(dest_path)
    
    weights = sparse_weights[sp]
    
    for key, val in weights.items():
        h, v = val.shape
        file_name = os.path.join(dest_path, "{}_h{}_v{}.txt".format(key, h, v))
        dump_to_txt(val, file_name)


dumped: sparse_weights/ts_95/sparsity_0.5/botl.0_h512_v13.txt
dumped: sparse_weights/ts_95/sparsity_0.5/botl.2_h256_v512.txt
dumped: sparse_weights/ts_95/sparsity_0.5/botl.4_h128_v256.txt
dumped: sparse_weights/ts_95/sparsity_0.5/topl.0_h1024_v479.txt
dumped: sparse_weights/ts_95/sparsity_0.5/topl.2_h1024_v1024.txt
dumped: sparse_weights/ts_95/sparsity_0.5/topl.4_h512_v1024.txt
dumped: sparse_weights/ts_95/sparsity_0.5/topl.6_h256_v512.txt
dumped: sparse_weights/ts_95/sparsity_0.5/topl.8_h1_v256.txt
dumped: sparse_weights/ts_95/sparsity_0.55/botl.0_h512_v13.txt
dumped: sparse_weights/ts_95/sparsity_0.55/botl.2_h256_v512.txt
dumped: sparse_weights/ts_95/sparsity_0.55/botl.4_h128_v256.txt
dumped: sparse_weights/ts_95/sparsity_0.55/topl.0_h1024_v479.txt
dumped: sparse_weights/ts_95/sparsity_0.55/topl.2_h1024_v1024.txt
dumped: sparse_weights/ts_95/sparsity_0.55/topl.4_h512_v1024.txt
dumped: sparse_weights/ts_95/sparsity_0.55/topl.6_h256_v512.txt
dumped: sparse_weights/ts_95/sparsity_0.55/t

In [ ]:
file_name

In [ ]:
ww = sparse_weights['0.5541957834992964']

In [ ]:
ww.keys()

In [ ]:

sparse

In [ ]:
round(0.432, 2)

In [ ]:
import matplotlib.pyplot as plt
total_el = 0
total_nz = 0

for w in weights.keys():
    sparsity_pattern = abs(weights[w]) == 0.
    n_element = sparsity_pattern.shape[0] * sparsity_pattern.shape[1]
    nz = sparsity_pattern.sum()
    
    total_el += n_element
    total_nz += nz
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(40, 20))
    fig.suptitle("Weights for {}, sparsity: {}".format(w, sparsity_pattern.sum() / n_element))
    ax1.imshow(sparsity_pattern, cmap="gray")
    ax2.plot(sparsity_pattern.sum(axis=0))
    ax3.plot(sparsity_pattern.sum(axis=1))

print('total sparsity: {}'.format(total_nz/total_el))

In [ ]:
import matplotlib.pyplot as plt
total_el = 0
total_nz = 0

for w in weights.keys():
    sparsity_pattern = abs(weights[w]) != 0.
    
    # Compute block size number of non zeros
    h, w = sparsity_pattern.shape
    
    if h >= 128 and w >= 128:
        num_nz_per_block = []
        blocksize = 16
        for i in range(h // blocksize):
            for j in range(w // blocksize):
                _block = sparsity_pattern[j*blocksize:(j+1)*blocksize, i*blocksize:(i+1)*blocksize]
                if _block.shape == (16, 16):
                    num_nz_per_block.append(1 - _block.sum() / (16*16))

        n_element = sparsity_pattern.shape[0] * sparsity_pattern.shape[1]
        nz = sparsity_pattern.sum()

        total_el += n_element
        total_nz += nz
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 20))
        fig.suptitle("Weights for {}, sparsity: {}".format(w, sparsity_pattern.sum() / n_element))
        ax1.imshow(sparsity_pattern, cmap="gray")
        ax2.plot(num_nz_per_block)


In [ ]:
sparsity_pattern.shape

In [ ]:
sparsity_pattern[0:16, 0:16].shape

In [ ]:
for i in range(256//16):
    print(i)

In [ ]:
def dump_to_txt(numpy_weights, path):
    # Should be 2-dim array
    assert len(numpy_weights.shape) == 2
    
    with open("sp_weight.txt", "w") as f:
        f.writelines(
            [" ".join([str(i) for i in l]) + "\n" for l in numpy_weights]            
        )

In [ ]:
path = './sparse_weights_0.95/'

In [ ]:
a = weights['botl.0'] == 0.00

In [ ]:
a.shape

In [ ]:
a.s